# Preparing news vectors

We want to use [word2vec] pre-computed word vectors to approximate
the semantic distance between user queries and dictionary definitions.

See Daniel Dacanay, Antti Arppe, and Atticus Harrigan, [Computational Analysis versus Human Intuition: A Critical Comparison of Vector Semantics with Manual Semantic Classification in the Context of Plains Cree][vecpaper1], in 
Proceedings of the 4th Workshop on the Use of Computational Methods in the Study of Endangered Languages; and 
Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean, [Efficient Estimation
of Word Representations in Vector Space][eewrvs], in Proceedings of Workshop at
ICLR, 2013.

[eewrvs]: http://arxiv.org/pdf/1301.3781.pdf
[word2vec]: https://code.google.com/archive/p/word2vec/
[vecpaper1]: https://computel-workshop.org/wp-content/uploads/2021/02/2021.computel-1.5.pdf

But first we are going to massage the precomputed vectors into an easier to use form.

There are a couple of things to do first:
  - Use a file format that’s faster to load
  - Save time and space by pruning keys we’ll never query for

## The upstream files

First, we’ll store the files in the `res/vector_models` directory.
Let’s make a variable that points at that.

In [33]:
import os
from pathlib import Path

# jupyter does not expose the filename of the notebook, and
# the kernel working directory appears to be the directory
# containing the first notebook opened in the jupyter session.
def find_project_root(target_filename='Pipfile'):
    """Walk upwards from current dir, looking for target_filename"""
    start_directory = directory = Path(os.getcwd())
    while directory.parent != directory:
        if (directory / target_filename).exists():
            return directory
        directory = directory.parent
    else:
        raise Exception(f'Could not find {target_filename!r} in any parent of {start_directory}')
    return directory

ROOT = find_project_root()
VECTOR_DIR = ROOT / 'CreeDictionary' / 'res' / 'vector_models'

The upstream `GoogleNews-vectors-negative300.bin.gz` file is not checked in here, so you’ll have to get it elsewhere.

In [34]:
!env BLOCK_SIZE="'1" ls -s $VECTOR_DIR

total 4,943,343,616
1,647,046,656 GoogleNews-vectors-negative300.bin.gz
  116,154,368 news_vectors.kv
1,800,003,584 news_vectors.kv.vectors.npy
1,380,139,008 news_vectors.kv.vectors.npy.zst


In [35]:
%%time

from gensim.models import KeyedVectors

vectors = VECTOR_DIR / 'GoogleNews-vectors-negative300.bin.gz'

wv = KeyedVectors.load_word2vec_format(vectors, binary=True)

CPU times: user 29.8 s, sys: 1.2 s, total: 31 s
Wall time: 31 s


It’s called a keyed vector because it maps keys to vectors.

If we run some basic stats, we see: there are 3 million, 300-dimensional vectors.

In [36]:
import numpy as np
def shortprint(a):
    with np.printoptions(threshold=10):
        print(a)
shortprint(wv['hello'])

[-0.05419922  0.01708984 -0.00527954 ... -0.36523438 -0.13769531
 -0.12890625]


In [37]:
len(wv['hello'])

300

In [38]:
len(wv.key_to_index)

3000000

We can query for similar concepts:

In [39]:
wv.similar_by_vector(wv['hello'])

[('hello', 1.0),
 ('hi', 0.6548984050750732),
 ('goodbye', 0.6399056315422058),
 ('howdy', 0.6310956478118896),
 ('goodnight', 0.5920578241348267),
 ('greeting', 0.5855877995491028),
 ('Hello', 0.5842196345329285),
 ("g'day", 0.5754078030586243),
 ('See_ya', 0.5688871741294861),
 ('ya_doin', 0.5643119812011719)]

And then the deep magic is that this vector model appears to capture semantic relationships.

Take the physics away from Einstein, add painting, and what do you get?

In [40]:
wv.similar_by_vector(wv['Einstein'] - wv['physics'] + wv['painting'])

[('painting', 0.6713024973869324),
 ('Picasso', 0.5790765285491943),
 ('Rembrandt', 0.5525932312011719),
 ('Guercino', 0.5175808072090149),
 ('paintings', 0.5061424970626831),
 ('Picasso_Monet', 0.5017005205154419),
 ('Balthus', 0.5001558661460876),
 ('Cezanne', 0.4969138503074646),
 ('Warhol', 0.49574798345565796),
 ('Vincent_Van_Gogh', 0.4956248998641968)]

### Faster file format

Let’s use the built-in gensim file format, which saves the vectors into a memory-mapping numpy array on disk.

In [41]:
import os

REDUCED_FILE = os.fspath(VECTOR_DIR / 'news_vectors.kv')

wv.save(REDUCED_FILE)

In [42]:
!env BLOCK_SIZE="'1" ls -s $VECTOR_DIR

total 6,743,343,104
1,647,046,656 GoogleNews-vectors-negative300.bin.gz
  116,154,368 news_vectors.kv
3,600,003,072 news_vectors.kv.vectors.npy
1,380,139,008 news_vectors.kv.vectors.npy.zst


In [43]:
%%time

wv = KeyedVectors.load(REDUCED_FILE, mmap='r')
with np.printoptions(threshold=10):
    print(wv['hello'])
wv.similar_by_vector(wv['hello'])

[-0.05419922  0.01708984 -0.00527954 ... -0.36523438 -0.13769531
 -0.12890625]
CPU times: user 2.44 s, sys: 892 ms, total: 3.34 s
Wall time: 1.17 s


[('hello', 1.0),
 ('hi', 0.6548984050750732),
 ('goodbye', 0.6399056315422058),
 ('howdy', 0.6310956478118896),
 ('goodnight', 0.5920578241348267),
 ('greeting', 0.5855877995491028),
 ('Hello', 0.5842196345329285),
 ("g'day", 0.5754078030586243),
 ('See_ya', 0.5688871741294861),
 ('ya_doin', 0.5643119812011719)]

In [44]:
%%time
wv.similar_by_vector(wv['hello'])

CPU times: user 1.72 s, sys: 459 ms, total: 2.18 s
Wall time: 147 ms


[('hello', 1.0),
 ('hi', 0.6548984050750732),
 ('goodbye', 0.6399056315422058),
 ('howdy', 0.6310956478118896),
 ('goodnight', 0.5920578241348267),
 ('greeting', 0.5855877995491028),
 ('Hello', 0.5842196345329285),
 ("g'day", 0.5754078030586243),
 ('See_ya', 0.5688871741294861),
 ('ya_doin', 0.5643119812011719)]

This file is *much* faster to load, ~1 second instead of 30 seconds, but it is also much larger.

#### Float precision

Interestingly, if we look inside the original file, we only started out with 16-bit floats, but they’re being stored as 32-bit ones. We can halve the file size by setting the data type correctly.

In [45]:
!zcat $VECTOR_DIR/GoogleNews-vectors-negative300.bin.gz \
    | head -c 300M | tail -c 256 | hexdump -C

00000000  93 be 00 00 14 3e 00 00  31 3e 00 00 41 3d 00 00  |.....>..1>..A=..|

gzip: 00000010  b8 3d 00 00 8f be 00 00  32 be 00 00 16 bf 00 00  |.=......2.......|
stdout: Broken pipe
00000020  08 be 00 00 c9 3e 00 00  9f be 00 00 5e 3c 00 00  |.....>......^<..|
00000030  30 be 00 00 58 bc 00 00  57 3e 00 00 8d 3e 00 00  |0...X...W>...>..|
00000040  a2 be 00 00 ee bd 00 00  e5 be 00 00 8f bd 00 00  |................|
00000050  21 be 00 00 3f be 00 00  74 3e 00 00 05 3f 00 00  |!...?...t>...?..|
00000060  2f 3e 00 00 dd bd 00 00  fa 3d 00 00 f6 3e 00 00  |/>.......=...>..|
00000070  16 be 00 00 5c 3e 00 00  3c 3e 00 00 2d be 00 00  |....\>..<>..-...|
00000080  9d bd 00 00 5a 3e 00 00  fe 3c 00 00 e6 3e 00 00  |....Z>...<...>..|
00000090  24 be 00 00 ec 3e 00 00  59 be 00 00 c9 3d 00 00  |$....>..Y....=..|
000000a0  f7 bd 00 00 07 bf 00 00  9d 3e 00 00 d8 bc 00 00  |.........>......|
000000b0  e9 3d 00 00 20 ba 00 00  4d 3e 00 00 b6 be 00 00  |.=.. ...M>......|
000000c0  ba bc 00 00 a7 

In [46]:
wv.vectors.dtype

dtype('float32')

In [47]:
wv2 = KeyedVectors.load(REDUCED_FILE, mmap='r')
wv2.vectors = wv2.vectors.astype('float16')
shortprint(wv2['hello'])

[-0.0542   0.01709 -0.00528 ... -0.3652  -0.1377  -0.1289 ]


But, sadly, doing so makes lookups take more than **15x** as long, going from a fraction of a second to multiple seconds. This is because modern CPUs do not generally have built-in 16-bit float operations.

In [48]:
%%time
wv2.similar_by_vector(wv2['hello'])

CPU times: user 3.79 s, sys: 906 ms, total: 4.69 s
Wall time: 2.37 s


[('hello', 1.0000041723251343),
 ('hi', 0.6548804640769958),
 ('goodbye', 0.639906108379364),
 ('howdy', 0.6311008334159851),
 ('goodnight', 0.5920441150665283),
 ('greeting', 0.5855898857116699),
 ('Hello', 0.5842040777206421),
 ("g'day", 0.5753953456878662),
 ('See_ya', 0.5688733458518982),
 ('ya_doin', 0.5643098950386047)]

In [49]:
%%time
wv.similar_by_vector(wv['hello'])

CPU times: user 1.79 s, sys: 479 ms, total: 2.27 s
Wall time: 153 ms


[('hello', 1.0),
 ('hi', 0.6548984050750732),
 ('goodbye', 0.6399056315422058),
 ('howdy', 0.6310956478118896),
 ('goodnight', 0.5920578241348267),
 ('greeting', 0.5855877995491028),
 ('Hello', 0.5842196345329285),
 ("g'day", 0.5754078030586243),
 ('See_ya', 0.5688871741294861),
 ('ya_doin', 0.5643119812011719)]

If we need to be careful with disk space, we could save the vectors on disk as float16 and then do `.astype('float32')` on load, which would only takes a few seconds. It would use more disk space, but may be much faster than dealing with a compressed file.

However, anything that’s not `mmap`ing a file gets risky in terms of memory use. A few gigs of data in memory isn’t a big deal for a server with lots of RAM, but (1) if the data isn’t all ready *before* the webserver forks worker processes, 10 copies of a few gigs of data adds up quickly, and (2) it could substantially increase the requirements for developer machines, which might not have many gigabytes of spare RAM.

So for now I think we’ll stick with the bigger file that can be processed more efficiently both in terms of RAM and CPU.

## Pruning keys

The file is still quite large. There’s probably a *lot* of stuff in there we will never, ever query for.

For example, what’s the millionth entry?

In [50]:
wv.index_to_key[1_000_000]

'Starwood_Hotels_HOT'

That’s not something we’ll ever query the dictionary for.

What are the top keys?

In [52]:
keys = list(wv.key_to_index.keys())

In [54]:
", ".join(keys[:100])

'</s>, in, for, that, is, on, ##, The, with, said, was, the, at, not, as, it, be, from, by, are, I, have, he, will, has, ####, his, an, this, or, their, who, they, but, $, had, year, were, we, more, ###, up, been, you, its, one, about, would, which, out, can, It, all, also, two, after, first, He, do, time, than, when, We, over, last, new, other, her, people, into, In, our, there, A, she, could, just, years, some, U.S., three, million, them, what, But, so, no, like, if, only, percent, get, did, him, game, back, because, now, #.#, before'

Right away we see that `#`—presumably a placeholder for a number—and `$` are common terms. What keys containing punctuation can we drop?

In [55]:
from collections import Counter
import string

[(char, f"{count:,}")
     for (char, count) in Counter(''.join(keys)).most_common()
     if char not in string.ascii_letters + string.digits][:30]

[('_', '2,881,208'),
 ('#', '261,534'),
 ('-', '202,358'),
 ('.', '187,835'),
 ('=', '29,938'),
 ('/', '24,442'),
 ("'", '17,088'),
 ('@', '10,956'),
 (':', '9,749'),
 ('é', '6,339'),
 (',', '5,517'),
 ('®', '4,268'),
 ('+', '2,530'),
 ('&', '2,493'),
 ('*', '2,350'),
 ('™', '2,206'),
 ('â', '1,647'),
 ('á', '1,483'),
 ('•', '1,428'),
 ('€', '1,324'),
 ('ó', '1,181'),
 ('í', '1,143'),
 ('ü', '1,125'),
 ('ñ', '1,045'),
 ('ö', '993'),
 ('è', '780'),
 ('ä', '599'),
 ('ç', '507'),
 ('е', '480'),
 ('ο', '464')]

We also see some duplication in terms of case; both “it” and “It” appear as keys.

In [58]:
wv.similar_by_key('It')

[('That', 0.8260787129402161),
 ('This', 0.8164012432098389),
 ("It'sa", 0.7155401706695557),
 ('But', 0.6960429549217224),
 ('Of_course', 0.6675450801849365),
 ('And', 0.6665104031562805),
 ('Certainly', 0.650726854801178),
 ("That'sa", 0.6421756148338318),
 ('Obviously', 0.6368812918663025),
 ('Actually', 0.6258060336112976)]

In [59]:
wv.similar_by_key('it')

[('that', 0.6775559782981873),
 ('something', 0.6162784695625305),
 ('just', 0.6107823848724365),
 ('actually', 0.5887327790260315),
 ('It', 0.5808414220809937),
 ('what', 0.5651708245277405),
 ('anyway', 0.5644350647926331),
 ('really', 0.5597794055938721),
 ('so', 0.5579650402069092),
 ('if', 0.5520145297050476)]

The distinction would definitely be useful for some purposes, but our dictionary lowercases all queries on input, so that would be lost on us.

In [200]:
import re
re_double_underscore = re.compile('.*_.*_.*')

def figure_out_keys_to_keep():
    # new_key, vector
    to_keep = {}
    
    # The original data does not seem to include frequencies, but we
    # assume that the keys are in frequency order, so we will see
    # the most common term first.
    for key in keys:        
        
        pruned_key = key.lower()
        if pruned_key in to_keep:
            continue
        
        # drop keys with unwanted punctuation
        if any(c in key for c in "$#.=/'@:,®+&*™•"):
            continue
        
        has_uppercase_char = key != key.lower()
        if has_uppercase_char:
            if '_' in key:
                continue
        
        # Skip items like “Dow_Jones_industrial”
        if re_double_underscore.match(key):
            continue
        
        to_keep[pruned_key] = wv[key]
    return to_keep

keys_to_keep = figure_out_keys_to_keep()

In [193]:
len(keys_to_keep)

930045

In [194]:
new_wv = KeyedVectors(vector_size=wv.vector_size)
new_keys = list(keys_to_keep.keys())
items = list(keys_to_keep.values())
new_wv.add_vectors(new_keys, items)

In [195]:
new_wv.similar_by_key('hello')

[('hullo', 0.6462694406509399),
 ('hey', 0.6447160840034485),
 ('bye_bye', 0.6369922757148743),
 ('hiya', 0.6357617378234863),
 ('goodbye', 0.6348499655723572),
 ('buh_bye', 0.6224703788757324),
 ('hi', 0.6207412481307983),
 ('ta_ta', 0.6033875942230225),
 ('yesss', 0.6024215817451477),
 ('ooooh', 0.5955002903938293)]

In [199]:
wv.similar_by_key('train')

[('trains', 0.8081232309341431),
 ('Train', 0.6729270219802856),
 ('commuter_train', 0.6523401737213135),
 ('locomotive', 0.6395583152770996),
 ('freight_train', 0.6207071542739868),
 ('railway', 0.6071822047233582),
 ('bus', 0.6067739725112915),
 ('Trains', 0.5963783264160156),
 ('rail', 0.5885170102119446),
 ('commuter_trains', 0.5821391344070435)]

In [198]:
new_wv.similar_by_key('train')

[('trains', 0.8054373860359192),
 ('commuter_train', 0.6855382919311523),
 ('rail', 0.6809316277503967),
 ('railway', 0.6646084189414978),
 ('locomotive', 0.6612517237663269),
 ('bus', 0.6542372107505798),
 ('freight_train', 0.6462224125862122),
 ('tram', 0.6416116952896118),
 ('commuter_trains', 0.6093850135803223),
 ('trolley', 0.6013067364692688)]

In [103]:
REDUCED_FILE2 = os.fspath(VECTOR_DIR / 'news_vectors_2.kv')

new_wv.save(REDUCED_FILE2)

In [104]:
!env BLOCK_SIZE="'1" ls -s $VECTOR_DIR

total 7,925,190,656
1,647,046,656 GoogleNews-vectors-negative300.bin.gz
   20,611,072 news_vectors_2.kv
1,116,057,600 news_vectors_2.kv.vectors.npy
  116,154,368 news_vectors.kv
3,600,003,072 news_vectors.kv.vectors.npy
1,425,317,888 news_vectors.kv.vectors.npy.zst


In [106]:
%%time

wv2 = KeyedVectors.load(REDUCED_FILE2, mmap='r')
with np.printoptions(threshold=10):
    print(wv['hello'])
wv2.similar_by_vector(wv2['hello'])

[-0.05419922  0.01708984 -0.00527954 ... -0.36523438 -0.13769531
 -0.12890625]
CPU times: user 951 ms, sys: 293 ms, total: 1.24 s
Wall time: 591 ms


[('hello', 1.0),
 ('hi', 0.6188791394233704),
 ('hiya', 0.5998829007148743),
 ('hey', 0.5955355167388916),
 ('hullo', 0.5693444609642029),
 ('oh', 0.5387828350067139),
 ('whadda', 0.537392795085907),
 ('konichiwa', 0.5368077754974365),
 ('hellooo', 0.5220615863800049),
 ('howzit', 0.520762026309967)]

In [107]:
%%time
wv2.similar_by_vector(wv2['hello'])

CPU times: user 542 ms, sys: 165 ms, total: 707 ms
Wall time: 50.2 ms


[('hello', 1.0),
 ('hi', 0.6188791394233704),
 ('hiya', 0.5998829007148743),
 ('hey', 0.5955355167388916),
 ('hullo', 0.5693444609642029),
 ('oh', 0.5387828350067139),
 ('whadda', 0.537392795085907),
 ('konichiwa', 0.5368077754974365),
 ('hellooo', 0.5220615863800049),
 ('howzit', 0.520762026309967)]

In [175]:
wv2.similar_by_key('greeting')

[('greetings', 0.7243465781211853),
 ('greet', 0.675140380859375),
 ('salutation', 0.6382814645767212),
 ('shaking_hands', 0.6265325546264648),
 ('greets', 0.6186001896858215),
 ('exchanging_greetings', 0.6183004975318909),
 ('salutations', 0.6101903915405273),
 ('greeted', 0.5959812998771667),
 ('welcoming', 0.57304847240448),
 ('shook_hands', 0.5713881850242615)]

In [178]:
wv.similar_by_key('Hello')

[('Hi', 0.6188791394233704),
 ('Hello_hello', 0.603535532951355),
 ('Goodbye', 0.6006848216056824),
 ('Hiya', 0.5998829007148743),
 ('Hey', 0.5955355167388916),
 ('Bruce_Springsteen_bellowed', 0.5867887139320374),
 ('hello', 0.5842196941375732),
 ('Welcome', 0.5704931616783142),
 ('Hullo', 0.5693444609642029),
 ('@_ESPN_Michelle', 0.5587143898010254)]

In [177]:
wv.similar_by_key('hello')

[('hi', 0.6548984050750732),
 ('goodbye', 0.6399056315422058),
 ('howdy', 0.6310956478118896),
 ('goodnight', 0.5920578241348267),
 ('greeting', 0.5855877995491028),
 ('Hello', 0.5842196345329285),
 ("g'day", 0.5754078030586243),
 ('See_ya', 0.5688871741294861),
 ('ya_doin', 0.5643119812011719),
 ('greet', 0.5636604428291321)]

In [179]:
wv2.similar_by_key('hello')

[('hi', 0.6188791394233704),
 ('hiya', 0.5998829007148743),
 ('hey', 0.5955355167388916),
 ('hullo', 0.5693444609642029),
 ('oh', 0.5387828350067139),
 ('whadda', 0.537392795085907),
 ('konichiwa', 0.5368077754974365),
 ('hellooo', 0.5220615863800049),
 ('howzit', 0.520762026309967),
 ('dear', 0.5133785605430603)]